In [0]:
import pandas as pd
import requests
import json

In [0]:
df=pd.read_csv("https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv", encoding="latin-1")

In [35]:
df.head()

,CBSA,NAME,POPESTIMATE2017,geoid
0,10180,"Abilene, TX",170219,31000US10180
1,10420,"Akron, OH",703505,31000US10420
2,10500,"Albany, GA",151434,31000US10500
3,10540,"Albany, OR",125047,31000US10540
4,10580,"Albany-Schenectady-Troy, NY",886188,31000US10580


3 Paramters are:
1. 50k-74.9k HH income
2. Age 18-34
3. Areas with large Hispanic populations

In [0]:
tableids = ['B01001', 'B03002','B19001']

In [47]:
tableidsstring = ','.join(tableids)
tableidsstring

'B01001,B03002,B19001'

In [0]:
geoid = '31000US' + df['CBSA'].astype(str)
df['geoid'] = geoid

In [0]:
# first half
geoidstring_part1 = ",".join(geoid[:int(len(geoid)/2)])
requesturl1 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring_part1)
loadedjson1 = requests.get(requesturl1)
parsedjson1 = json.loads(loadedjson1.text)

# second half
geoidstring_part2 = ",".join(geoid[int(len(geoid)/2):])
requesturl2 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring_part2)
loadedjson2 = requests.get(requesturl2)
parsedjson2 = json.loads(loadedjson2.text)

# combine two json together
Everything = dict(parsedjson1['data'], **parsedjson2['data'])

In [52]:
Everything['31000US10180']['B03002']['estimate']

{'B03002001': 174006.0,
 'B03002002': 133580.0,
 'B03002003': 110633.0,
 'B03002004': 13466.0,
 'B03002005': 648.0,
 'B03002006': 3491.0,
 'B03002007': 0.0,
 'B03002008': 578.0,
 'B03002009': 4764.0,
 'B03002010': 706.0,
 'B03002011': 4058.0,
 'B03002012': 40426.0,
 'B03002013': 21001.0,
 'B03002014': 1075.0,
 'B03002015': 0.0,
 'B03002016': 461.0,
 'B03002017': 27.0,
 'B03002018': 16003.0,
 'B03002019': 1859.0,
 'B03002020': 871.0,
 'B03002021': 988.0}

In [0]:
HH_Income = []
Pop_by_age = []
Pop_by_Hispanic = []
for i, row in df.iterrows():
    Income = Everything[df['geoid'][i]]['B19001']['estimate']['B19001011']# B19001011 $50,000 to $59,999
    + Everything[df['geoid'][i]]['B19001']['estimate']['B19001012'] # B19001012 $60,000 to $74,999
    HH_Income.append(Income)
    Pop = Everything[df['geoid'][i]]['B01001']['estimate']['B01001007']# B01001007 18 and 19 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001008']# B01001008 20 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001009']# B01001009 21 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001010']# B01001010 22 to 24 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001011']# B01001011 25 to 29 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001012']# B01001012 30 to 34 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001031']# B01001031 18 and 19 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001032']# B01001032 20 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001033']# B01001033 21 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001034']# B01001034 22 to 24 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001035']# B01001035 25 to 29 Years
    + Everything[df['geoid'][i]]['B01001']['estimate']['B01001036']# B01001036 30 to 34 Years
    Pop_by_age.append(Pop)
    Hispanic = Everything[df['geoid'][i]]['B03002']['estimate']['B03002012']# B03002012 Hispanic or Latino
    Pop_by_Hispanic.append(Hispanic)

df['Pop_by_Income'] = HH_Income
df['Pop_by_Age'] = Pop_by_age
df['Pop_by_Hispanic'] = Pop_by_Hispanic

In [56]:
print(df.loc[df['Pop_by_Income'] == max(df['Pop_by_Income'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


1. New York-Newark-Jersey City, NY-NJ-PA has the highest count of 50k-74.9k audience

In [57]:
print(df.loc[df['Pop_by_Age'] == max(df['Pop_by_Age'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


2. New York-Newark-Jersey City, NY-NJ-PA has the highest count of ages 18-34 audience

In [58]:
print(df.loc[df['Pop_by_Hispanic'] == max(df['Pop_by_Hispanic'])]['NAME'])

210    Los Angeles-Long Beach-Anaheim, CA
Name: NAME, dtype: object


3. Los Angeles-Long Beach-Anaheim, CA has the highest count of hispanic population.

In [0]:
HH_Income_Percentage = []
Pop_by_age_Percentage = []
Pop_by_Hispanic_Percentage = []
for i, row in df.iterrows():
    Percentage_Income = df['Pop_by_Income'][i]/max(df['Pop_by_Income'])
    Percentage_Age = df['Pop_by_Age'][i]/max(df['Pop_by_Age'])
    Percentage_Hispanic = df['Pop_by_Hispanic'][i]/max(df['Pop_by_Hispanic'])
    HH_Income_Percentage.append(Percentage_Income)
    Pop_by_age_Percentage.append(Percentage_Age)
    Pop_by_Hispanic_Percentage.append(Percentage_Hispanic)

df['HH_Income_Percentage'] = HH_Income_Percentage
df['Pop_by_age_Percentage'] = Pop_by_age_Percentage
df['Pop_by_Hispanic_Percentage'] = Pop_by_Hispanic_Percentage

mean_percentage_score = []
for i, row in df.iterrows():
    Mean = (df['HH_Income_Percentage'][i] + df['Pop_by_age_Percentage'][i]+ df['Pop_by_Hispanic_Percentage'][i])/3
    mean_percentage_score.append(Mean)
df['mean_percentage_score'] = mean_percentage_score

In [60]:
print(df.loc[df['mean_percentage_score'] == max(df['mean_percentage_score'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


4. New York-Newark-Jersey City, NY-NJ-PA has the highest mean percentage score across all three categories